# Laboratorio 3.4: Bloc de notas del estudiante

## Información general

Este laboratorio es una continuación de los laboratorios guiados del Módulo 3. 

En este laboratorio, dividirá los datos en tres conjuntos de datos separados:

- *Conjunto de entrenamiento*: se utilizará para entrenar el modelo.
- *Conjunto de validación*: se utilizará durante el entrenamiento para validar el modelo.
- *Conjunto de prueba*: este se retendrá y se utilizará para producir métricas después de entrenar el modelo. Utilizará este conjunto de datos en un próximo laboratorio.

Con los datos divididos, entrenará un modelo XGBoost utilizando Amazon SageMaker.


## Introducción al escenario empresarial

Usted trabaja para un proveedor de atención médica y desea mejorar la detección de anomalías en pacientes ortopédicos. 

Tiene la tarea de resolver este problema mediante el aprendizaje automático. Tiene acceso a un conjunto de datos que contiene seis características biomecánicas y un objetivo de *normal* o *anormal*. Puede utilizar este conjunto de datos para entrenar un modelo de aprendizaje automático para predecir si un paciente va a tener una anomalía.


## Acerca de este conjunto de datos

Este conjunto de datos biomédicos fue creado por el Dr. Henrique da Mota durante un período de residencia médica en el Grupo de Investigación Aplicada en Ortopedia (GARO, Group of Applied Research in Orthopaedics) del Centre Médico-Chiurgical de Réadaptation des Massues, Lyon, Francia. Los datos se han organizado en dos tareas de clasificación diferentes, pero relacionadas. 

La primera tarea consiste en clasificar a los pacientes como pertenecientes a una de tres categorías: 

- *Normal* (100 pacientes)
- *Hernia de disco* (60 pacientes)
- *Espondilolistesis* (150 pacientes)

Para la segunda tarea, las categorías *Hernia de disco* y *Espondilolistesis* se fusionaron en una sola categoría etiquetada como *anormal*. Por lo tanto, la segunda tarea consiste en clasificar a los pacientes como pertenecientes a una de dos categorías: *Normal* (100 pacientes) o *Anormal* (210 pacientes).


## Información de atributos:

Cada paciente está representado en el conjunto de datos por seis atributos biomecánicos que se derivan de la forma y orientación de la pelvis y la columna lumbar (en este orden): 

- Incidencia pélvica
- Inclinación pélvica
- Ángulo de lordosis lumbar
- Inclinación del sacro
- Radio pélvico
- Grado de espondilolistesis

La siguiente convención se utiliza para las etiquetas de clase: 
- Hernia Disco (HD)
- Espondilolistesis (EL)
- Normal (NO) 
- Anormal (AN)


Para obtener más información acerca de este conjunto de datos, consulte [Página web del conjunto de datos de la columna vertebral] (http://archive.ics.uci.edu/ml/datasets/Vertebral+Column).


## Atribuciones del conjunto de datos

Este conjunto de datos se obtuvo de:
Dua, D. y Graff, C. (2019). Repositorio de aprendizaje automático de la UCI (http://archive.ics.uci.edu/ml). Irvine, CA: Universidad de California, Escuela de Ciencias de la Información e Informática.


# Configuración del laboratorio
Dado que esta solución se divide en varios laboratorios del módulo, debe ejecutar las siguientes celdas para poder cargar los datos:

## Importación de los datos

Al ejecutar las celdas siguientes, los datos se importarán y estarán listos para su uso. 

**Nota:** Las siguientes celdas representan los pasos clave en los laboratorios anteriores.

In [1]:
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff
import boto3

In [2]:
f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

In [3]:
data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

In [4]:
class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

# Paso 1: Explorar los datos
Comenzará con un recordatorio rápido de los datos del conjunto de datos.

Para sacar el máximo provecho de este laboratorio, lea detenidamente las instrucciones y el código antes de ejecutar las celdas. ¡Tómese el tiempo para experimentar!

Primero, use **forma** para examinar el número de filas y columnas.

In [5]:
df.shape

(310, 7)

A continuación, obtenga una lista de las columnas.

In [6]:
df.columns

Index(['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis', 'class'],
      dtype='object')

Puede ver las seis características biomecánicas y que la columna objetivo se denomina *clase*.


# Paso 2: Preparar los datos

Para este laboratorio, debe dividir los datos en tres conjuntos de datos.

Una búsqueda en Internet mostrará muchas maneras diferentes de dividir conjuntos de datos. Muchos ejemplos de código que puede encontrar dividirán el conjunto de datos en *objetivo* y *características*. Luego, dividirán cada uno de esos dos conjuntos de datos en tres subconjuntos, lo que da como resultado un total de seis conjuntos de datos para realizar un seguimiento.

## Traslado de la posición de la columna objetivo

XGBoost requiere que los datos de entrenamiento estén en un solo archivo. El archivo debe tener el valor objetivo como la primera columna. 

Obtenga la columna objetivo y muévala a la primera posición.

In [7]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

Debería ver que la **clase** es ahora la primera columna.

In [8]:
df.columns

Index(['class', 'pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis'],
      dtype='object')

## División de los datos

Comenzará dividiendo el conjunto de datos en dos conjuntos de datos. Utilizará un conjunto de datos para el entrenamiento y volverá a dividir el otro conjunto de datos para utilizarlo con la validación y las pruebas.

Utilizará la función *división_entrenamiento_prueba* de la *biblioteca scikit-learn*, que es una biblioteca de aprendizaje automático gratuita para Python. Tiene muchos algoritmos y funciones útiles, como la que va a utilizar. 

- Para obtener más información acerca de la función, consulte la [Documentación de división de entrenamiento_prueba](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). 
 - Para obtener más información sobre scikit-learn, consulte la [guía scikit-learn] (https://scikit-learn.org/stable/)

Debido a que no tiene muchos datos, desea asegurarse de que los conjuntos de datos divididos contienen una cantidad representativa de cada clase. Por lo tanto, deberá usar el modificador *estratificar*. Finalmente, usará un número aleatorio para poder repetir las divisiones.

In [9]:
from sklearn.model_selection import train_test_split
train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])

A continuación, divida el conjunto de datos *probar_y_validar* en dos partes iguales.

In [10]:
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

Examine los tres conjuntos de datos.

In [11]:
print(train.shape)
print(test.shape)
print(validate.shape)

(248, 7)
(31, 7)
(31, 7)


Ahora, compruebe la distribución de las clases.

In [12]:
print(train['class'].value_counts())
print(test['class'].value_counts())
print(validate['class'].value_counts())

1    168
0     80
Name: class, dtype: int64
1    21
0    10
Name: class, dtype: int64
1    21
0    10
Name: class, dtype: int64


## Carga de los datos a Amazon S3

XGBoost cargará los datos para el entrenamiento de Amazon Simple Storage Service (Amazon S3). Por lo tanto, debe escribir los datos en un archivo de valores separados por comas (CSV) y luego cargar el archivo en Amazon S3.

Comience configurando algunas variables en el bucket de S3 y, a continuación, cree una función para cargar el archivo CSV en Amazon S3. Puede volver a usar esta función.

Primero, explore la función.

Observe la siguiente línea:

`dataframe.to_csv(csv_buffer, header=False, index=False)`

Esta línea escribe el DataFrame de pandas (que se pasó a la función) en el búfer de E/S, que se denomina *búfer_csv*. Usted utiliza un búfer dado que no es necesario escribir el archivo en forma local.

Para evitar que se escriban los encabezados de las columnas, utilice `header=False`. Para evitar que se produzca la salida del índice de pandas, use `index=False`.

Para escribir el búfer_csv en Amazon S3 como un objeto, utilice la operación `put` en `object`, que es una propiedad de `bucket`.


In [13]:
bucket='c47992a689653l4129474t1w374228752902-labbucket-17il0lfebcov2'

prefix='lab3'

train_file='vertebral_train.csv'
test_file='vertebral_test.csv'
validate_file='vertebral_validate.csv'

import os

s3_resource = boto3.Session().resource('s3')
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False)
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

Utilice la función que creó para cargar los tres conjuntos de datos.

In [14]:
upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

# Paso 3: Entrenar el modelo

Ahora que los datos están en Amazon S3, puede entrenar un modelo. 

El primer paso es obtener el URI del contenedor XGBoost.

In [15]:
import boto3
from sagemaker.image_uris import retrieve
container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')


A continuación, debe establecer algunos *hiperparámetros* para el modelo. Dado que es la primera vez que está entrenando el modelo, puede usar algunos valores para comenzar.

In [16]:
hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic"}

Utilice la función **estimador** para configurar el modelo. Aquí hay algunos parámetros de interés:

- **recuento_de_instancia**: define cuántas instancias se utilizarán para el entrenamiento. Utilizará *una* instancia.
- **tipo_de_instancia**: define el tipo de instancia para el entrenamiento. En este caso, es *ml.m4.xlarge*.


In [17]:
import sagemaker
s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=sagemaker.estimator.Estimator(container,
                                       sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

El estimador necesita *canales* para ingresar datos en el modelo. Para el entrenamiento, se usarán el *canal_de_entrenamiento* y el *canal_de_validación*.

In [18]:
train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

Al ejecutar **ajustar** se entrenará el modelo.

**Nota:** Este proceso puede tardar hasta 5 minutos.

In [19]:
xgb_model.fit(inputs=data_channels, logs=False)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-05-18-22-27-53-428



2023-05-18 22:27:53 Starting - Starting the training job....
2023-05-18 22:28:19 Starting - Preparing the instances for training................
2023-05-18 22:29:49 Downloading - Downloading input data.....
2023-05-18 22:30:19 Training - Downloading the training image........
2023-05-18 22:31:04 Training - Training image download completed. Training in progress.....
2023-05-18 22:31:30 Uploading - Uploading generated training model..
2023-05-18 22:31:41 Completed - Training job completed


Después de completar el entrenamiento, estará listo para probar y evaluar el modelo. Sin embargo, hará pruebas y validación en laboratorios posteriores.

# ¡Felicitaciones!

Ha completado este laboratorio y ahora puede terminarlo siguiendo las instrucciones de la guía del laboratorio.